In [32]:
# Import modules
from logger import setup_custom_logger
import sys
import pandas as pd    
import cbsodata
import datetime

In [33]:
# Setup of logger
try:
    logger = setup_custom_logger("E001_BBP")
    logger.info('starting')
except:
    logger.exception('logger could not be loaded')
    raise

2021-03-18 13:26:33 INFO     E001_BBP starting
2021-03-18 13:26:33 INFO     E001_BBP starting
2021-03-18 13:26:33 INFO     E001_BBP starting


INFO:E001_BBP:starting


In [34]:
try:
    # Get current date information
    now = datetime.datetime.now()
    
    logger.info('datetime loaded')
    yearMin = now.year-4
    yearMax = now.year
    
    # Values you can load
    # now.year
    # now.month
    # now.day
    # now.hour
    # now.minute
except:
    logger.exception('datetime could not be loaded')
    logger.info('set yearmin to a default value')
    # Set default values for fallback
    yearMin = 2010
    yearMax = 2030
    raise

2021-03-18 13:26:33 INFO     E001_BBP datetime loaded
2021-03-18 13:26:33 INFO     E001_BBP datetime loaded
2021-03-18 13:26:33 INFO     E001_BBP datetime loaded


INFO:E001_BBP:datetime loaded


In [35]:
# Dataset 84106NED
dataset_id = '84106NED'

# Table definitions

# SoortMutaties: A045299 = Volume, t.o.v. zelfde periode vorig jaar
# BrutoBinnenlandsProduct_2: waarde BBP

In [36]:
try:
    logger.info(f'Retrieve data from dataset {dataset_id}')
    data = pd.DataFrame(cbsodata.get_data(
    f'{dataset_id}',
    filters=f"substring(Perioden,0,4) ge '{yearMin}' and SoortMutaties eq 'A045299'",
    select=["Perioden", "BrutoBinnenlandsProduct_2"]))
except:
    logger.exception('error loading data from CBS Statline')
    raise

2021-03-18 13:26:35 INFO     E001_BBP Retrieve data from dataset 84106NED
2021-03-18 13:26:35 INFO     E001_BBP Retrieve data from dataset 84106NED
2021-03-18 13:26:35 INFO     E001_BBP Retrieve data from dataset 84106NED


INFO:E001_BBP:Retrieve data from dataset 84106NED


In [37]:
# Remove quaterly and yearly data
try:
    data = data[data["Perioden"].str.contains("kwartaal")==True]
    data = data[data['Perioden'].map(len) > 4 ]
except:
    logger.exception('Perioden filter could not be applied')
    raise

In [38]:
# Rename columns
try:
    data = data.rename(columns={
    "BrutoBinnenlandsProduct_2": "Volume, t.o.v zelfde periode vorig jaar in procenten,
})

except:
    logger.exeption('Columns could not be renamed')
    raise

In [39]:
# Export dataFrame to Excel file
try:
    data.to_excel("output/E001_BBP.xlsx")  
except:
    logger.exception('dataFrame could not be exported to output folder')